
# Classificação de Risco de Câncer com Rede Neural Artificial

Trabalho prático da disciplina de **Inteligência Artificial** do mestrado.

Objetivo: a partir da tabela `classificacao_risco_cancer_varios_valores.csv`, realizar:

1. **Limpeza e preparação dos dados**.
2. **Construção de um modelo de Rede Neural Artificial** para classificação.
3. **Validação cruzada k-fold** (com estratificação).
4. Obter **acurácia média na validação cruzada ≥ 0.70**.
5. Documentar todas as etapas realizadas.

Neste notebook, a variável-alvo considerada é a coluna **`Level`**, que representa o nível de risco de câncer (baixa, moderada, alta etc.).


## 1. Importação de bibliotecas

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import io

## 2. Carregamento do conjunto de dados

In [ ]:

uploader = widgets.FileUpload(accept='.csv', multiple=False)
display(uploader)


In [ ]:
# Ler o CSV enviado
if len(uploader.value) == 0:
    raise ValueError("Nenhum arquivo enviado. Faça o upload acima.")

file_content = list(uploader.value.values())[0]['content']
df = pd.read_csv(io.BytesIO(file_content))

df.head()


## 3. Análise exploratória inicial

In [ ]:
# Informações gerais
print("Formato do dataset:", df.shape)
print("\nTipos de dados:")
print(df.dtypes)

print("\nValores ausentes por coluna:")
print(df.isna().sum())

# Estatísticas descritivas das variáveis numéricas
df.describe(include=[np.number])


In [ ]:
# Visualizar estatísticas das variáveis categóricas
df.describe(include=['object'])



## 4. Limpeza e preparação dos dados

Passos adotados nesta etapa:

1. **Remoção de colunas irrelevantes** para a predição (por exemplo, identificadores).
2. **Tratamento de valores ausentes (`NaN`)**:
   - Para variáveis numéricas: imputação pela **mediana**.
   - Para variáveis categóricas: imputação pelo **valor mais frequente**.
3. **Tratamento de strings que representam valores faltantes**, como `"NaN"`, `"nan"`, `"None"`, `"?"`, etc.
4. Definição da variável **alvo** (`y`) como a coluna `Level` e das variáveis **explicativas** (`X`) como o restante.
5. Remoção de linhas com alvo (`Level`) ausente, pois não podem ser usadas no treinamento.


In [ ]:
# Copiar o dataframe para evitar efeitos colaterais
dados = df.copy()

# Substituir strings que representam valores ausentes por NaN real
valores_nulos_str = ["NaN", "nan", "NONE", "None", "?", "null", "NULL", " "]
dados = dados.replace(valores_nulos_str, np.nan)

# Remover a coluna de identificador (não contém informação preditiva)
if "Patient Id" in dados.columns:
    dados = dados.drop(columns=["Patient Id"])

# Definir alvo (y) e preditores (X)
if "Level" not in dados.columns:
    raise ValueError("A coluna 'Level' não foi encontrada no dataset. Verifique o nome da variável-alvo.")

# Remover linhas sem rótulo (Level)
dados = dados.dropna(subset=["Level"])

X = dados.drop(columns=["Level"])
y = dados["Level"]

print("Formato de X:", X.shape)
print("Formato de y:", y.shape)

# Conferir distribuição das classes
print("\nDistribuição das classes em 'Level':")
print(y.value_counts(normalize=True))


## 5. Separação de variáveis numéricas e categóricas

In [ ]:
# Identificar colunas numéricas e categóricas
colunas_numericas = X.select_dtypes(include=[np.number]).columns.tolist()
colunas_categoricas = X.select_dtypes(include=["object"]).columns.tolist()

print("Variáveis numéricas:", colunas_numericas)
print("Variáveis categóricas:", colunas_categoricas)



## 6. Pipeline de pré-processamento

Será utilizado um **ColumnTransformer** com dois blocos principais:

- **Numérico**:
  - `SimpleImputer(strategy="median")`
  - `StandardScaler()`

- **Categórico**:
  - `SimpleImputer(strategy="most_frequent")`
  - `OneHotEncoder(handle_unknown="ignore")`

Esse pré-processamento será acoplado a um classificador de rede neural (`MLPClassifier`) por meio de um **Pipeline** do scikit-learn.


In [ ]:
# Pipeline para variáveis numéricas
transformador_numerico = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline para variáveis categóricas
transformador_categorico = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combinar em um ColumnTransformer
preprocessador = ColumnTransformer(
    transformers=[
        ("num", transformador_numerico, colunas_numericas),
        ("cat", transformador_categorico, colunas_categoricas)
    ]
)

preprocessador



## 7. Modelo de Rede Neural Artificial (MLPClassifier)

Utilizaremos o `MLPClassifier` do scikit-learn, que implementa uma **Rede Neural Multicamada** (MLP).

Configuração inicial sugerida:

- `hidden_layer_sizes=(50, 30)` → duas camadas escondidas com 50 e 30 neurônios;
- `activation="relu"`;
- `solver="adam"`;
- `max_iter=500`;
- `random_state=42` para reprodutibilidade.

Esse classificador será colocado no final de um `Pipeline` que inclui o pré-processamento definido na seção anterior.


In [ ]:
# Definição do classificador (rede neural)
mlp = MLPClassifier(
    hidden_layer_sizes=(50, 30),
    activation="relu",
    solver="adam",
    max_iter=500,
    random_state=42
)

# Pipeline completo: pré-processamento + classificador
modelo = Pipeline(steps=[
    ("preprocessamento", preprocessador),
    ("classificador", mlp)
])

modelo



## 8. Validação cruzada k-fold

Será utilizada validação cruzada **k-fold estratificada** com:

- `n_splits = 10` (10 folds);
- `shuffle = True` para embaralhar os dados;
- `random_state = 42` para reprodutibilidade.

A métrica utilizada será a **acurácia** (`scoring="accuracy"`).

O objetivo é que a **acurácia média** seja **maior ou igual a 0.70** (≥ 70%). 


In [ ]:
# Definir o esquema de validação cruzada estratificada
k = 10
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Avaliar o modelo com cross_val_score
scores = cross_val_score(
    modelo,
    X,
    y,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1  # usa todos os núcleos disponíveis, se suportado
)

print(f"Acurácias por fold ({k}-fold):")
print(scores)

print("\nAcurácia média: {:.4f}".format(scores.mean()))
print("Desvio padrão: {:.4f}".format(scores.std()))



## 9. Discussão dos resultados

- A tabela de acurácias por fold mostra o desempenho do modelo em cada partição da validação cruzada.
- A **acurácia média** é o valor de referência principal.
- O **desvio padrão** indica o quão estável é o desempenho do modelo entre os folds.

Caso a acurácia média seja **inferior a 0.70**, é possível tentar melhorias, por exemplo:

- Ajustar a arquitetura da rede neural (número de neurônios, número de camadas).
- Ajustar hiperparâmetros como `learning_rate_init`, `alpha` (regularização), `max_iter` etc.
- Realizar algum balanceamento de classes, se houver forte desbalanceamento.
- Testar seleção de variáveis ou engenharia de atributos.

Se a acurácia média for **≥ 0.70**, o requisito do exercício está atendido.


## 10. Experimentos adicionais (opcional)


Nesta seção, podem ser adicionados testes extras, como:

- Outras configurações da rede neural (diferentes `hidden_layer_sizes`).
- Comparação com outros classificadores (árvore de decisão, florestas aleatórias etc.),
  apenas para fins de análise, ainda que a solução final deva ser baseada em **rede neural artificial**.


In [ ]:
# Exemplo (opcional): testar outra arquitetura de rede neural
# Descomente e ajuste conforme necessário

# mlp2 = MLPClassifier(
#     hidden_layer_sizes=(100, 50),
#     activation="relu",
#     solver="adam",
#     max_iter=800,
#     random_state=42
# )
#
# modelo2 = Pipeline(steps=[
#     ("preprocessamento", preprocessador),
#     ("classificador", mlp2)
# ])
#
# scores2 = cross_val_score(
#     modelo2,
#     X,
#     y,
#     cv=cv,
#     scoring="accuracy",
#     n_jobs=-1
# )
#
# print("Acurácias por fold (modelo 2):", scores2)
# print("Acurácia média (modelo 2):", scores2.mean())
